In [1]:
import torch
import numpy as np
import argparse
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import re
import torch.optim as optim
import torch.nn.init as initialization
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from itertools import product as cartesian_prod
import random
from scipy import spatial

Contants

In [2]:
args={}
kwargs={}
args['batch_size']=32
args['input_shape'] = 32*32*3
args['epochs']=30  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=3e-4 #Learning rate is how fast it will descend. 
args['eps']=1e-7
args['depth'] = 4
args['width'] = [64, 64, 64]
args['classes'] = 2
 
args['cuda']=False
args['log_interval']=10

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cpu')

Load the dataset

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

print(trainset)
print()

y_train = np.array(trainset.targets)

index = np.where(np.in1d(y_train, [0,1]))[0]
print("Selected train datapoints:", len(index))

trainset_01 = torch.utils.data.Subset(trainset, index)

# index = np.where(y_train == 0)

trainloader = torch.utils.data.DataLoader(trainset_01, batch_size=args['batch_size'],
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

y_test = np.array(testset.targets)

index = np.where(np.in1d(y_test, [0,1]))[0]
print("Selected test datapoints:", len(index))

testset_01 = torch.utils.data.Subset(testset, index)

testloader = torch.utils.data.DataLoader(testset_01, batch_size=args['batch_size'],
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

Selected train datapoints: 10000
Files already downloaded and verified
Selected test datapoints: 2000


In [ ]:
np.unique(y_train), np.unique(y_test)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

Train and validation function

In [5]:

def train(model, epoch, train_loader, lossFn, optimizer):
  model.train()
  loss = 0
  net_loss = 0
  correct = 0
  
  for batch_idx, (data, target) in enumerate(train_loader):
      # if args['cuda']:
      #     data, target = data.cuda(), target.cuda()
      
      #Reshape the input image to 1D vector
      data = data.reshape(-1, args['input_shape']).to(device)
      target = target.unsqueeze(1).float().to(device)

      #This will zero out the gradients for this batch. 
      optimizer.zero_grad()

      #Run the model on the train data
      _, output = model(data)
      
      # Calculate the loss
      loss = lossFn(output, target)
      net_loss += loss.data

      #dloss/dx for every Variable 
      loss.backward()

      #to do a one-step update on our parameter.
      optimizer.step()

      pred = (output > 0.5).float()

      correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

      
  #Print out the loss
  print('Train Epoch: {} \tLoss: {:.6f}, Accuracy: {}/{} ({:.0f}%)'.format(
          epoch, net_loss/len(train_loader.dataset), correct, len(train_loader.dataset),
      100. * correct / len(train_loader.dataset)))

def validate(model, validation_loader, lossFn):

  model.eval()
  validation_loss = 0
  correct = 0

  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
    for data, target in validation_loader:
      # if args['cuda']:
      #     data, target = data.cuda(), target.cuda()

      #Reshape the input image to 1D vector
      data = data.reshape(-1, args['input_shape']).to(device)
      target = target.unsqueeze(1).float().to(device)

      #Run the model on the validation data and get its predictions
      _, output = model(data)
      
      #Calculate the loss
      validation_loss += lossFn(output, target).data.item()  #sum up batch loss

      pred = (output > 0.5).float()

      correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()  #find the number of correct predictions

  print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      validation_loss/len(validation_loader.dataset), correct, len(validation_loader.dataset),
      100. * correct / len(validation_loader.dataset)))

ReLU NN

In [ ]:
class ReluNetwork(nn.Module):
  def __init__(self, depth, width, classes, input_shape):

    super(ReluNetwork, self).__init__()
    self.depth = depth
    self.width = width

    setattr(self,'R1', nn.Linear(input_shape, width[0]))
    for i in range(depth - 2):
        setattr(self,'R{}'.format(i+2), nn.Linear(width[i], width[i+1]))

    if classes == 2:
      self.output = nn.Linear(width[depth-2], 1) 
    
    else:
      self.output = nn.Linear(width[depth-2], classes) 


  def forward(self, x, **kwargs):
    
    R1 = F.relu(getattr(self,'R1')(x))
    for i in range(self.depth-2):
      R1 = F.relu(getattr(self,'R{}'.format(i+2))(R1))

    Q = self.output(R1)
    Z = F.sigmoid(Q)

    return Q, Z

In [ ]:
args['input_shape']

3072

Adam(1e-4)

In [ ]:
model = ReluNetwork(args['depth'], args['width'], args['classes'], args['input_shape'])
print(model)
print()
lossFn = nn.BCELoss()

if args['cuda']:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=args['lr'])

for epoch in range(1, args['epochs'] + 1):
    train(model, epoch, trainloader, lossFn, optimizer)
    validate(model, testloader, lossFn)

ReluNetwork(
  (R1): Linear(in_features=3072, out_features=64, bias=True)
  (R2): Linear(in_features=64, out_features=64, bias=True)
  (R3): Linear(in_features=64, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)

Train Epoch: 1 	Loss: 0.013436, Accuracy: 7986/10000 (80%)

Validation set: Average loss: 0.0103, Accuracy: 1707/2000 (85%)

Train Epoch: 2 	Loss: 0.010228, Accuracy: 8552/10000 (86%)

Validation set: Average loss: 0.0092, Accuracy: 1725/2000 (86%)

Train Epoch: 3 	Loss: 0.008944, Accuracy: 8757/10000 (88%)

Validation set: Average loss: 0.0087, Accuracy: 1764/2000 (88%)

Train Epoch: 4 	Loss: 0.007836, Accuracy: 8989/10000 (90%)

Validation set: Average loss: 0.0086, Accuracy: 1776/2000 (89%)

Train Epoch: 5 	Loss: 0.006960, Accuracy: 9104/10000 (91%)

Validation set: Average loss: 0.0084, Accuracy: 1777/2000 (89%)

Train Epoch: 6 	Loss: 0.006194, Accuracy: 9214/10000 (92%)

Validation set: Average loss: 0.0089, Accuracy: 1765/2000 

DLGN

In [6]:
eps, beta = 0.1, 4

In [7]:
#Class implementing the soft ReLU activation function
class SoftGate(nn.Module):
    def __init__(self, **kwargs):
        super(SoftGate, self).__init__()

    def build(self, input_shape):
        super(SoftGate, self).build(input_shape)  

    def forward(self, x):
        activation = (1 + eps)*torch.sigmoid(beta*x)
        return activation

    def compute_output_shape(self, input_shape):
        return input_shape


In [8]:
class GaluConstantNPVWithNPFBias(nn.Module):
  def __init__(self, depth, width, classes, input_size):

    super(GaluConstantNPVWithNPFBias, self).__init__()

    self.depth = depth
    self.width = width

    setattr(self,'NPF_L1', nn.Linear(input_size, width[0]))  #NPF network's layer 1

    setattr(self,'NPV_L1', nn.Linear(input_size, width[0], bias=False))

    for i in range(depth - 2):
      setattr(self,'NPF_L{}'.format(i+2), nn.Linear(width[i], width[i+1]))

      setattr(self,'NPV_L{}'.format(i+2), nn.Linear(width[i], width[i+1], bias=False))

    # Output layer (in NPV network)
    if classes == 2:
      setattr(self,'NPV_L{}'.format(depth), nn.Linear(width[depth-2], 1, bias=False))
    
    else:
      setattr(self,'NPV_L{}'.format(depth), nn.Linear(width[depth-2], classes, bias=False)) 
    
      
  def forward(self, x):

    Zf = getattr(self,'NPF_L1')(x)  #Linear activations(So Qf = Zf) of layer 1 of NPF network
    G = SoftGate()(Zf)  #Gating values

    Qv = getattr(self,'NPV_L1')(torch.Tensor(np.ones(x.shape)).to(device))  #Pre-activations of layer 1 of NPV network with constant input
    Zv = torch.mul(G, Qv)  #Gating value(from NPF) times Pre-Activations of layer 1 of NPV network is the Activations for layer 1 of NPV network

    # Repeat the above computations for all the hidden layers
    for i in range(self.depth-2):
      Zf = getattr(self,'NPF_L{}'.format(i+2))(Zf)
      G = SoftGate()(Zf)

      Qv = getattr(self,'NPV_L{}'.format(i+2))(Zv)
      Zv = torch.mul(G, Qv)

    Q_out = getattr(self,'NPV_L{}'.format(self.depth))(Zv)
    Z_out = F.sigmoid(Q_out)

    return Q_out, Z_out

In [ ]:
decoupled_const_NPV_model_with_NPF_bias = GaluConstantNPVWithNPFBias(args['depth'], args['width'], args['classes'], args['input_shape'])
print(decoupled_const_NPV_model_with_NPF_bias)
print()

GaluConstantNPVWithNPFBias(
  (NPF_L1): Linear(in_features=3072, out_features=64, bias=True)
  (NPV_L1): Linear(in_features=3072, out_features=64, bias=False)
  (NPF_L2): Linear(in_features=64, out_features=64, bias=True)
  (NPV_L2): Linear(in_features=64, out_features=64, bias=False)
  (NPF_L3): Linear(in_features=64, out_features=64, bias=True)
  (NPV_L3): Linear(in_features=64, out_features=64, bias=False)
  (NPV_L4): Linear(in_features=64, out_features=1, bias=False)
)



In [ ]:
decoupled_const_NPV_model_with_NPF_bias = decoupled_const_NPV_model_with_NPF_bias.to(device)
lossFn = nn.BCELoss().to(device)

optimizer = optim.Adam(decoupled_const_NPV_model_with_NPF_bias.parameters(), lr=args['lr'], eps=args['eps'])

for epoch in range(1, args['epochs'] + 1):
    train(decoupled_const_NPV_model_with_NPF_bias, epoch, trainloader, lossFn, optimizer)
    validate(decoupled_const_NPV_model_with_NPF_bias, testloader, lossFn)

Train Epoch: 1 	Loss: 0.012510, Accuracy: 8092/10000 (81%)

Validation set: Average loss: 0.0104, Accuracy: 1697/2000 (85%)

Train Epoch: 2 	Loss: 0.009571, Accuracy: 8634/10000 (86%)

Validation set: Average loss: 0.0098, Accuracy: 1740/2000 (87%)

Train Epoch: 3 	Loss: 0.008357, Accuracy: 8858/10000 (89%)

Validation set: Average loss: 0.0095, Accuracy: 1745/2000 (87%)

Train Epoch: 4 	Loss: 0.007447, Accuracy: 8998/10000 (90%)

Validation set: Average loss: 0.0086, Accuracy: 1765/2000 (88%)

Train Epoch: 5 	Loss: 0.006363, Accuracy: 9145/10000 (91%)

Validation set: Average loss: 0.0098, Accuracy: 1755/2000 (88%)

Train Epoch: 6 	Loss: 0.005910, Accuracy: 9230/10000 (92%)

Validation set: Average loss: 0.0095, Accuracy: 1760/2000 (88%)

Train Epoch: 7 	Loss: 0.005193, Accuracy: 9314/10000 (93%)

Validation set: Average loss: 0.0099, Accuracy: 1761/2000 (88%)

Train Epoch: 8 	Loss: 0.004296, Accuracy: 9442/10000 (94%)

Validation set: Average loss: 0.0105, Accuracy: 1772/2000 (89%)



In [ ]:
for name,params in decoupled_const_NPV_model_with_NPF_bias.named_parameters():
  print(f"{name} \t {params.shape}")

NPF_L1.weight 	 torch.Size([64, 3072])
NPF_L1.bias 	 torch.Size([64])
NPV_L1.weight 	 torch.Size([64, 3072])
NPF_L2.weight 	 torch.Size([64, 64])
NPF_L2.bias 	 torch.Size([64])
NPV_L2.weight 	 torch.Size([64, 64])
NPF_L3.weight 	 torch.Size([64, 64])
NPF_L3.bias 	 torch.Size([64])
NPV_L3.weight 	 torch.Size([64, 64])
NPV_L4.weight 	 torch.Size([1, 64])


Store weights of all layers

In [ ]:
torch.save(decoupled_const_NPV_model_with_NPF_bias.state_dict(), './Constant NPV Model for CIFAR With Bias for NPF')

Load the model

In [9]:
decoupled_const_NPV_model_with_NPF_bias = GaluConstantNPVWithNPFBias(args['depth'], args['width'], args['classes'], args['input_shape'])
decoupled_const_NPV_model_with_NPF_bias.load_state_dict(torch.load('./Constant NPV Model for CIFAR With Bias for NPF'))
decoupled_const_NPV_model_with_NPF_bias.eval()

GaluConstantNPVWithNPFBias(
  (NPF_L1): Linear(in_features=3072, out_features=64, bias=True)
  (NPV_L1): Linear(in_features=3072, out_features=64, bias=False)
  (NPF_L2): Linear(in_features=64, out_features=64, bias=True)
  (NPV_L2): Linear(in_features=64, out_features=64, bias=False)
  (NPF_L3): Linear(in_features=64, out_features=64, bias=True)
  (NPV_L3): Linear(in_features=64, out_features=64, bias=False)
  (NPV_L4): Linear(in_features=64, out_features=1, bias=False)
)

Function to get the weights of NPV network

In [10]:
def get_params(model, str, bias=False):
  weights = []
  biases = []

  for i in range(args['depth']):
    layer_weight = model.state_dict()[str+'_L{}.weight'.format(i+1)]
    weights.append(layer_weight)

    if bias:
      layer_bias = model.state_dict()[str+'_L{}.bias'.format(i+1)]
      biases.append(layer_bias)

  if bias:
    return weights, biases
    
  return weights

Function to get the effective weights of NPF network

In [11]:
def get_NPF_effective_weights(model, bias=False):

  effective_weights = []
  effective_biases = []
  
  for i in range(args['depth']-1):
      curr_weight = model.state_dict()['NPF_L{}.weight'.format(i+1)]
      if i==0:
          effective_weights.append(curr_weight)
          if bias:
            curr_bias = model.state_dict()['NPF_L{}.bias'.format(i+1)]
            effective_biases.append(curr_bias)
      else:
          effective_weights.append(torch.matmul(curr_weight,effective_weights[-1]))
          if bias:
            curr_bias = model.state_dict()['NPF_L{}.bias'.format(i+1)]
            effective_biases.append(torch.matmul(curr_weight, effective_biases[-1])+curr_bias)

  if bias:
    return effective_weights, effective_biases
    
  return effective_weights
  # effective_weights is a list of size depth-1

Calculate importance value for all paths

In [12]:
def get_path_value(weights, paths):
  
  output_layer = args['depth']-1
  path_values = dict()

  for path in paths:
    
    importance_value = np.ones((args['input_shape']))   # initialize to constant(input to NPV network is all 1s)
    prev_nodes = range(args['input_shape'])   # nodes to be considered in the previous layer

    # print(path)

    # loop to compute importance value of current path
    for layer,node in enumerate(path):

      weight = weights[layer][node][prev_nodes].cpu()
    
      importance_value = np.dot(weight, importance_value)
      prev_nodes = node

    
    weight = weights[output_layer][:, prev_nodes].cpu()
      
    importance_value = importance_value *  weight # consider all output nodes for each path

    # store the importance value and the path
    path_values[path] = importance_value

  return path_values

Generate all possible paths

In [16]:
widths = args['width']
print(widths)
print(len(widths))

[64, 64, 64]
3


In [17]:
paths = list(cartesian_prod(*[range(x) for x in widths]))
print(paths[:20])
print()
print(paths[-20:])

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 0, 5), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 0, 9), (0, 0, 10), (0, 0, 11), (0, 0, 12), (0, 0, 13), (0, 0, 14), (0, 0, 15), (0, 0, 16), (0, 0, 17), (0, 0, 18), (0, 0, 19)]

[(63, 63, 44), (63, 63, 45), (63, 63, 46), (63, 63, 47), (63, 63, 48), (63, 63, 49), (63, 63, 50), (63, 63, 51), (63, 63, 52), (63, 63, 53), (63, 63, 54), (63, 63, 55), (63, 63, 56), (63, 63, 57), (63, 63, 58), (63, 63, 59), (63, 63, 60), (63, 63, 61), (63, 63, 62), (63, 63, 63)]


In [18]:
NPV_weights = get_params(decoupled_const_NPV_model_with_NPF_bias, str='NPV', bias=False)

In [ ]:
len(NPV_weights)

4

In [ ]:
NPV_weights[0].shape

torch.Size([64, 3072])

In [ ]:
output_layer = args['depth']-1
NPV_weights[output_layer][ :, 2].shape

torch.Size([1])

In [19]:
path_values = get_path_value(NPV_weights, paths)

In [ ]:
print(len(paths))
print(len(path_values))

262144
262144


In [ ]:
print("Path: 1-2-3:")
print(path_values[(1,2,3)])
print()
print("Path: 29-11-27:")
print(path_values[(29,11,27)])

Path: 1-2-3:
tensor([-0.0024])

Path: 29-11-27:
tensor([-0.0008])


Sort the paths by L1 norm of importance value vector

In [20]:
sorted_keys = sorted(path_values, key = lambda dict_key: abs(path_values[dict_key]), reverse=True)
for k in sorted_keys[:10]:
  print(k, path_values[k])

(7, 63, 59) tensor([0.3382])
(7, 57, 23) tensor([0.3111])
(27, 33, 25) tensor([0.2871])
(7, 32, 17) tensor([-0.2832])
(7, 57, 42) tensor([-0.2805])
(7, 51, 59) tensor([-0.2788])
(27, 33, 59) tensor([0.2778])
(38, 10, 37) tensor([0.2746])
(7, 63, 17) tensor([0.2706])
(61, 22, 37) tensor([-0.2696])


In [21]:
NPF_effective_weights, NPF_effective_biases = get_NPF_effective_weights(decoupled_const_NPV_model_with_NPF_bias, bias=True)

In [ ]:
print(NPF_effective_biases[0][0])
print(NPF_effective_biases[0][0].item())

tensor(-0.0392)
-0.03919714689254761


Function to compute pre-activations using pathwise decomposition

In [ ]:
weight = NPF_effective_weights[2][5].cpu().detach().numpy().reshape((1,args['input_shape']))
weight.shape

(1, 3072)

In [22]:
def pathwise_decomposition_network(paths, path_vectors, instance, NPF_effective_weights, NPF_effective_biases, n = None, threshold = None):

  pre_activations = 0   # since number of classes = 1
  i = 1

  for path in paths:
    active = 1
    
    for layer, node in enumerate(path):
      weight = NPF_effective_weights[layer][node].cpu().detach().numpy().reshape(1,args['input_shape'])
      # print(weight.shape)
      # print(instance.T.shape)
      # print(np.dot(weight, instance.T).reshape(-1,) + 5)
      gating_signal = np.dot(weight, instance.T).T + NPF_effective_biases[layer][node].cpu().detach().numpy()
      active *= SoftGate()(torch.tensor(gating_signal))


    if threshold is not None:
      # print(active.shape)
      if active.item() >= threshold:
        pre_activations = pre_activations + active * path_values[path]
        i += 1

        if i==n:
          break

    else:
      pre_activations = pre_activations + active * path_values[path]

  
  return pre_activations

In [23]:
def get_path_truncation_outputs(paths, path_vectors, NPF_effective_weights, NPF_effective_biases, data, target, n = None, threshold=None):
  
  correct = 0
  outputs = np.empty((0,1))

  if threshold is None:
    Q_out = pathwise_decomposition_network(paths, path_vectors, data, NPF_effective_weights, NPF_effective_biases, n, threshold)
    Z_out = F.sigmoid(torch.tensor(Q_out))
    outputs = Q_out

  else:
    Z_out = np.empty((0,1))  # since only 1 value is output by the network
    for instance in data:
      # print('Data shape:', data.shape)
      # print("Instance shape: ", instance.shape)
      Q_out = pathwise_decomposition_network(paths, path_vectors, instance, NPF_effective_weights, NPF_effective_biases, n, threshold)
      Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])
      outputs = np.vstack([outputs, Q_out])

    Z_out = torch.tensor(Z_out)


  # print("Predictions")
  # print(Q_out.shape)
  # # print(Q_out)
  # print()
  

  pred = (Z_out > 0.5).float()  #get the index of the max log-probability which is the predicted class
  # print("Class label")
  # print(pred.shape)
  # # print(pred)
  # print()

  correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
  # print("Accuracy")
  # print(correct)
  # print(len(data))

  accuracy = correct/len(data)

  return accuracy, outputs

In [24]:
def get_dataset(loader):
  X = np.empty((0, args['input_shape']))
  Y = np.empty((0))

  for data, target in loader:
    X = np.concatenate((X, data.cpu().detach().numpy().reshape(-1, args['input_shape'])), axis=0)
    Y = np.concatenate((Y, target.cpu().detach().numpy()), axis=0)

  return X, Y

In [25]:
X_train, Y_train = get_dataset(trainloader)
print(X_train.shape, Y_train.shape)

(10000, 3072) (10000,)


In [ ]:
print(Y_train)

[0. 0. 1. ... 1. 1. 0.]


In [ ]:
np.unique(Y_train)

array([0., 1.])

In [ ]:
print(len(X_train), len([X_train[5]]))

10000 1


In [26]:
def avg_cosine_similarity(a, b):
  avg = 0
  for (i,j) in zip(a,b):
    avg += 1 - spatial.distance.cosine(i,j)

  avg /= len(a)

  return avg

Experiments on Train data

5 samples

In [ ]:
accuracy_path, outputs_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, X_train[:5], torch.tensor(Y_train[:5]))
print()
print("Accuracy:", accuracy_path)
print("Output shape:", outputs_path.shape)


Accuracy: tensor(1.)
Output shape: torch.Size([5, 1])


<ipython-input-35-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
X = torch.tensor(X_train[:5], dtype=torch.float)
X = X.to(device)
# print(X.is_cuda)
Q, Z = decoupled_const_NPV_model_with_NPF_bias(X)

In [ ]:
pred = (Z > 0.5).float()
print(pred)
correct = 0
correct += pred.eq(torch.tensor(Y_train[:5]).data.view_as(pred)).long().cpu().sum()
print("Correct: ", correct)

tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.]])
Correct:  tensor(5)


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_path)

0.9999999915576391

All samples

In [ ]:
accuracy_path, outputs_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print()
print("Accuracy:", accuracy_path)
print("Output shape:", outputs_path.shape)


Accuracy: tensor(0.9754)
Output shape: torch.Size([10000, 1])


<ipython-input-35-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [27]:
X = torch.tensor(X_train[:10000], dtype=torch.float)
X = X.to(device)
# print(X.is_cuda)
Q, Z = decoupled_const_NPV_model_with_NPF_bias(X)
pred = (Z > 0.5).float()
correct = 0
correct += pred.eq(torch.tensor(Y_train[:10000]).data.view_as(pred)).long().cpu().sum()
accuracy = correct/10000
print("Accuracy: ", accuracy)

Accuracy:  tensor(0.9754)


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_path)

0.9999999891920099

1. Static Path model

Top 50000 path model

In [ ]:
accuracy_50000_path, outputs_50000_path = get_path_truncation_outputs(sorted_keys[:50000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print("Accuracy:", accuracy_50000_path)
print("Output shape:", outputs_50000_path.shape)

Accuracy: tensor(0.9450)
Output shape: torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_50000_path)

0.9263999897990014

Top 40000 path model

In [ ]:
accuracy_40000_path, outputs_40000_path = get_path_truncation_outputs(sorted_keys[:40000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print("Accuracy:", accuracy_40000_path)
print("Output shape:", outputs_40000_path.shape)

Accuracy: tensor(0.9429)
Output shape: torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_40000_path)

0.9209999898259612

Top 30000 path model

In [ ]:
accuracy_30000_path, outputs_30000_path = get_path_truncation_outputs(sorted_keys[:30000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print(accuracy_30000_path)
print(outputs_30000_path.shape)

tensor(0.9489)
torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_30000_path)

0.9261999898056893

Top 20000 path

In [ ]:
accuracy_20000_path, outputs_20000_path = get_path_truncation_outputs(sorted_keys[:20000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print(accuracy_20000_path)
print(outputs_20000_path.shape)

tensor(0.9358)
torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_20000_path)

0.8979999900138042

Top 10000 path model

In [ ]:
accuracy_10000_path, outputs_10000_path = get_path_truncation_outputs(sorted_keys[:10000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print(accuracy_10000_path)
print(outputs_10000_path.shape)

tensor(0.9097)
torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_10000_path)

0.8325999904911612

Top 1000 Path model

In [ ]:
accuracy_1000_path, outputs_1000_path = get_path_truncation_outputs(sorted_keys[:1000], path_values, NPF_effective_weights, NPF_effective_biases, X_train, torch.tensor(Y_train))
print(accuracy_1000_path)
print(outputs_1000_path.shape)

tensor(0.7158)
torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_1000_path)

0.4479999935246936

Top 100 Path model

In [ ]:
accuracy_100_path, outputs_100_path = get_path_truncation_outputs(sorted_keys[:100], path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                  X_train, torch.tensor(Y_train))
print(accuracy_100_path)
print(outputs_100_path.shape)

tensor(0.6131)
torch.Size([10000, 1])


<ipython-input-21-f98378ccf83d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_100_path)

0.23859999534683565

2. Dynamic Path model

Top 5000 path(active)

In [ ]:
# accuracy_5000_active_path, outputs_5000_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
#                                                                                   X_train, torch.tensor(Y_train), n=5000, threshold=0.5)
# print(accuracy_5000_active_path)
# print(outputs_5000_active_path.shape)

In [ ]:
# avg_cosine_similarity(Q_all.cpu().detach().numpy(), outputs_5000_active_path)

Top 1000 path(active)

In [ ]:
accuracy_1000_active_path, outputs_1000_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                                  X_train, torch.tensor(Y_train), n=1000, threshold=0.5)
print(accuracy_1000_active_path)
print(outputs_1000_active_path.shape)

<ipython-input-46-4729100ac689>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_1000_active_path)

Top 100 path(active)

In [28]:
accuracy_100_active_path, outputs_100_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                                X_train, torch.tensor(Y_train), n=100, threshold=0.5)
print(accuracy_100_active_path)
print(outputs_100_active_path.shape)

<ipython-input-23-4729100ac689>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])


tensor(0.7163)
(10000, 1)


In [29]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_100_active_path)

0.4461999936828317

Experiments on Validation data

In [30]:
X_val, Y_val = get_dataset(testloader)
print(X_val.shape, Y_val.shape)

(2000, 3072) (2000,)


Verifying pathwise decomposition model

In [ ]:
accuracy_path, outputs_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print()
print("Accuracy:", accuracy_path)
print("Output shape:", outputs_path.shape)


Accuracy: tensor(0.8755)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [31]:
X = torch.tensor(X_val, dtype=torch.float)
X = X.to(device)
# print(X.is_cuda)
Q, Z = decoupled_const_NPV_model_with_NPF_bias(X)
pred = (Z > 0.5).float()
correct = 0
correct += pred.eq(torch.tensor(Y_val).data.view_as(pred)).long().cpu().sum()
accuracy = correct/10000
print("Accuracy: ", accuracy)

Accuracy:  tensor(0.1751)


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_path)

0.9999999892558786

1. Static path model

Top 50000 paths

In [ ]:
accuracy_50000_path, outputs_50000_path = get_path_truncation_outputs(sorted_keys[:50000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_50000_path)
print("Output shape:", outputs_50000_path.shape)

Accuracy: tensor(0.8560)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_50000_path)

0.9049999900673623

Top 40000 path

In [ ]:
accuracy_40000_path, outputs_40000_path = get_path_truncation_outputs(sorted_keys[:40000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_40000_path)
print("Output shape:", outputs_40000_path.shape)

Accuracy: tensor(0.8545)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_40000_path)

0.8859999902007399

Top 30000 path

In [ ]:
accuracy_30000_path, outputs_30000_path = get_path_truncation_outputs(sorted_keys[:30000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_30000_path)
print("Output shape:", outputs_30000_path.shape)

Accuracy: tensor(0.8585)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_30000_path)

0.8819999902721978

Top 20000 path

In [ ]:
accuracy_20000_path, outputs_20000_path = get_path_truncation_outputs(sorted_keys[:20000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_20000_path)
print("Output shape:", outputs_20000_path.shape)

Accuracy: tensor(0.8475)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_20000_path)

0.8419999904970926

Top 10000 path

In [ ]:
accuracy_10000_path, outputs_10000_path = get_path_truncation_outputs(sorted_keys[:10000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_10000_path)
print("Output shape:", outputs_10000_path.shape)

Accuracy: tensor(0.8255)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_10000_path)

0.7739999912000993

Top 1000 path

In [ ]:
accuracy_1000_path, outputs_1000_path = get_path_truncation_outputs(sorted_keys[:1000], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_1000_path)
print("Output shape:", outputs_1000_path.shape)

Accuracy: tensor(0.6795)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_1000_path)

0.4139999940314131

Top 100 path

In [ ]:
accuracy_100_path, outputs_100_path = get_path_truncation_outputs(sorted_keys[:100], path_values, NPF_effective_weights, NPF_effective_biases, X_val, torch.tensor(Y_val))
print("Accuracy:", accuracy_100_path)
print("Output shape:", outputs_100_path.shape)

Accuracy: tensor(0.6050)
Output shape: torch.Size([2000, 1])


<ipython-input-20-4729100ac689>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = F.sigmoid(torch.tensor(Q_out))


In [ ]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_100_path)

0.23299999552926348

2. Dynamic Path model

Top 2500 active paths


In [36]:
accuracy_2500_active_path, outputs_2500_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                                  X_val, torch.tensor(Y_val), n=2500, threshold=0.5)
print(accuracy_2500_active_path)
print(outputs_2500_active_path.shape)

<ipython-input-23-4729100ac689>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])


tensor(0.8335)
(2000, 1)


In [37]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_2500_active_path)

0.8219999907225084

Top 1000 active paths

In [34]:
accuracy_1000_active_path, outputs_1000_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                                  X_val, torch.tensor(Y_val), n=1000, threshold=0.5)
print(accuracy_1000_active_path)
print(outputs_1000_active_path.shape)

<ipython-input-23-4729100ac689>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])


tensor(0.8100)
(2000, 1)


In [35]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_1000_active_path)

0.7369999914009807

Top 100 active paths

In [32]:
accuracy_100_active_path, outputs_100_active_path = get_path_truncation_outputs(sorted_keys, path_values, NPF_effective_weights, NPF_effective_biases, 
                                                                                  X_val, torch.tensor(Y_val), n=100, threshold=0.5)
print(accuracy_100_active_path)
print(outputs_100_active_path.shape)

<ipython-input-23-4729100ac689>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z_out = np.vstack([Z_out, F.sigmoid(torch.tensor(Q_out))])


tensor(0.6735)
(2000, 1)


In [33]:
avg_cosine_similarity(Q.cpu().detach().numpy(), outputs_100_active_path)

0.3899999943404594